In [1]:
#---01---
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()

print(f[1])
print(list(f))
print(20 in f)
print(15 in f)

10
[0, 10, 20]
True
False


In [2]:
#---02---
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit)
                       for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [12]:
#---03---
import collections
from random import shuffle
from frenchdeck import FrenchDeck

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit)
                       for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


deck = FrenchDeck()
shuffle(deck)


TypeError: 'FrenchDeck' object does not support item assignment

In [11]:
#---04---
import collections
from random import shuffle

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit)
                       for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card

deck = FrenchDeck()
shuffle(deck)

print(deck[:5])

[Card(rank='K', suit='hearts'), Card(rank='J', suit='clubs'), Card(rank='Q', suit='hearts'), Card(rank='3', suit='hearts'), Card(rank='3', suit='clubs')]


In [16]:
#---05---
from collections.abc import MutableSequence
import collections
from random import shuffle

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit)
                       for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)

deck2 = FrenchDeck2()
shuffle(deck2)
print(deck2[:5])

[Card(rank='6', suit='diamonds'), Card(rank='4', suit='spades'), Card(rank='Q', suit='diamonds'), Card(rank='K', suit='hearts'), Card(rank='10', suit='diamonds')]


In [17]:
#---06---
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""
        pass

    @abc.abstractmethod
    def pick(self):
        """
        Remove item at random, returning it.
        This method should raise LookupError when the instance is empty.
        """
        pass

    def loaded(self):
        """Return True if there's at least 1 item, False otherwise."""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [26]:
#---07---
import random
from tombola import Tombola

class BingoCage(Tombola):
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def load(self, iterable):
        self._items.extend(iterable)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):
        return self.pick()

In [28]:
#---08---
import random
from tombola import Tombola

class LotteryBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(sorted(self._balls))

In [ ]:
#---08---
from random import randrange
from tombola import Tombola

@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(sorted(self))